<a href="https://colab.research.google.com/github/haesungpyun/LLM_KD/blob/Lin/(250_finetuning)t5_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#설치 라이브러리

# colab에서 GPU 사용시, 다음 코드를 먼저 실행해야 sacrebleu 설치 오류 안 남.

# import locale
# def getpreferredencoding(do_setlocale = True):
#     return "UTF-8"
# locale.getpreferredencoding = getpreferredencoding


#!pip install datasets SentencePiece sacrebleu

## Load model, tokenizer

In [2]:
import torch
from transformers import T5Tokenizer, AdamW
from torch.utils.data import DataLoader, Dataset

from transformers import T5Tokenizer, T5ForConditionalGeneration
from datasets import load_dataset


dataset = load_dataset("bbaaaa/iwslt14-de-en")

# Tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the

## Load data & split

In [17]:
train= dataset['train']
test= dataset['test']
seed_value = 42
train = train.shuffle(seed=seed_value)
test = test.shuffle(seed=seed_value)

train_50= train.select(range(50))          #for baseline
train_250= train.select(range(250))        #for baseline
unlabel1=train.select(range(10000,20000))  #for MCKD
unlabel2=train.select(range(20000,30000))  #for MCKD

n=100
test_n= test.select(range(n))

## Training

In [4]:
torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device(type='cuda')

In [5]:
from time import time
import numpy as np

In [6]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
start= time()

# DataLoader for batch processing
batch_size = 16
train_loader = DataLoader(train_250['translation'], batch_size=batch_size)
#eval_loader = DataLoader(dataset['evaluation']['translation'], batch_size=batch_size)

optimizer = AdamW(model.parameters(), lr=3e-4) ##lr from huggingface tip

# Training loop
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()

for epoch in range(num_epochs):
    now=time()
    print(f"Epoch {epoch + 1}/{num_epochs}, time: {np.round(now-start,3)//60}분 {np.round((now-start,3)%60)}초")

    for batch in train_loader:
        de_inputs = torch.tensor(tokenizer(batch['de'], padding=True).input_ids).to(device)
        en_labels = torch.tensor(tokenizer(batch['en'], padding=True).input_ids).to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=de_inputs, labels=en_labels)
        loss = outputs.loss #cpu: 1분에 1배치 (16), cuda: 1배치당 0.1초
        loss.backward()
        optimizer.step()

    print(f"Loss: {loss.item()}")

# Save the fine-tuned model
model.save_pretrained('./fine_tuned_t5_model')

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10, time: 0.0분 5.903초
Loss: 1.301015019416809
Epoch 2/10, time: 0.0분 16.476초
Loss: 0.8126190900802612
Epoch 3/10, time: 0.0분 24.027초
Loss: 0.5265286564826965
Epoch 4/10, time: 0.0분 31.657초
Loss: 0.4145004451274872
Epoch 5/10, time: 0.0분 39.351초
Loss: 0.28334397077560425
Epoch 6/10, time: 0.0분 47.098초
Loss: 0.20695315301418304
Epoch 7/10, time: 0.0분 54.875초
Loss: 0.16533568501472473
Epoch 8/10, time: 1.0분 2.6430000000000007초
Loss: 0.12700384855270386
Epoch 9/10, time: 1.0분 10.385000000000005초
Loss: 0.08077181875705719
Epoch 10/10, time: 1.0분 18.051000000000002초
Loss: 0.06468721479177475


In [18]:
# DataLoader for batch processing
batch_size = 16
test_loader = DataLoader(test_n['translation'], batch_size=batch_size) #100
model.eval()
print("냐옹!")

냐옹!


## Inference & BLEU

In [8]:
outputs= []
ref=[]
i=0

start= time()

with torch.no_grad():
    for batch in test_loader:
        i+=1
        now= time()
        print(f"batch {i} / {n//batch_size+1}, time: {np.round(now-start,3)//60}분 {np.round(now-start,3)%60}초")

        de_inputs = torch.tensor(tokenizer(batch['de'], padding=True).input_ids).to(device)
        ref.append(batch['en'])
        output= model.generate(de_inputs)
        for sentence in output:
          outputs.append(tokenizer.decode(sentence, skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


batch 1 / 7, time: 0.0분 0.001초
batch 2 / 7, time: 0.0분 0.57초
batch 3 / 7, time: 0.0분 1.069초
batch 4 / 7, time: 0.0분 1.525초
batch 5 / 7, time: 0.0분 1.978초
batch 6 / 7, time: 0.0분 2.413초
batch 7 / 7, time: 0.0분 2.845초


In [9]:
#output이 아니라 outputs 입니다

outputs[0]

'bg: an energy breach is the allerwichtigest.'

In [10]:
# batched list -> list

reference = []
for sublist in ref:
    for item in sublist:
        reference.append(item)

In [13]:
import sacrebleu

bleu = sacrebleu.corpus_bleu(outputs, [reference])
print(f"SacreBLEU score: {bleu.score}")

SacreBLEU score: 11.687752266823585


실제 번역 내용:

In [15]:
outputs[:3]

['bg: an energy breach is the allerwichtigest.',
 'am morgen, before the game.',
 "they've seen this week, but it's simply a bad experiment."]

In [16]:
reference[:3]

['bg: an energy breakthrough is the most important thing.',
 "it's morning, before the match.",
 "you've seen recently, corn to ethanol is just a bad experiment."]